In [1]:

import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from PIL import Image

# Define transforms for both training and testing
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Define the paths to the train and test data directories
train_data_dir = 'Dataset/train/' 
test_data_dir = 'Dataset/test/'    

# Load train and test datasets using ImageFolder
train_dataset = ImageFolder(root=train_data_dir, transform=transform)
test_dataset = ImageFolder(root=test_data_dir, transform=transform)

# Define batch size for dataloaders
batch_size = 32

# Create dataloaders for train and test datasets
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
print(train_loader.dataset.classes)
print(test_loader.dataset.classes)

# Load pre-trained ResNet model
resnet = models.resnet50(pretrained=True)

# Freeze all layers except the last fully connected layer
for param in resnet.parameters():
    param.requires_grad = False

# Modify the last fully connected layer for binary classification (2 output classes)
num_ftrs = resnet.fc.in_features
resnet.fc = nn.Linear(num_ftrs, len(train_loader.dataset.classes))

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(resnet.parameters(), lr=0.001, momentum=0.9)

# Training loop
num_epochs = 10
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
resnet.to(device)

for epoch in range(num_epochs):
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()

        outputs = resnet(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 10 == 9:  # Print every 10 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 10))
            running_loss = 0.0

print('Finished Training')

['13020220048', '13020220049', '13020220050', '13020220051', '13020220052', '13020220053', '13020220054', '13020220056', '13020220148', '13020220152', '13020220155', '13020220157', '13020220159', '13020220160', '13020220167', '13020220173', '13020220303', '13020220304', '13020220305']
['13020220048', '13020220049', '13020220050', '13020220051', '13020220052', '13020220053', '13020220054', '13020220056', '13020220148', '13020220152', '13020220155', '13020220157', '13020220159', '13020220160', '13020220167', '13020220173', '13020220303', '13020220304', '13020220305']


d:\Data Science\FRP\AttendanceFaceRecognition\.venv\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
d:\Data Science\FRP\AttendanceFaceRecognition\.venv\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


[1,    10] loss: 2.969
[1,    20] loss: 2.799
[1,    30] loss: 2.627
[1,    40] loss: 2.445
[1,    50] loss: 2.337
[1,    60] loss: 2.181
[1,    70] loss: 2.041
[1,    80] loss: 1.928
[1,    90] loss: 1.833
[1,   100] loss: 1.759
[1,   110] loss: 1.660
[1,   120] loss: 1.735
[1,   130] loss: 1.598
[1,   140] loss: 1.544
[1,   150] loss: 1.538
[2,    10] loss: 1.391
[2,    20] loss: 1.373
[2,    30] loss: 1.374
[2,    40] loss: 1.240
[2,    50] loss: 1.178
[2,    60] loss: 1.209
[2,    70] loss: 1.183
[2,    80] loss: 1.148
[2,    90] loss: 1.183
[2,   100] loss: 1.103
[2,   110] loss: 1.091
[2,   120] loss: 1.095
[2,   130] loss: 0.957
[2,   140] loss: 1.023
[2,   150] loss: 0.963
[3,    10] loss: 0.936
[3,    20] loss: 0.900
[3,    30] loss: 0.940
[3,    40] loss: 0.920
[3,    50] loss: 0.908
[3,    60] loss: 0.924
[3,    70] loss: 0.806
[3,    80] loss: 0.816
[3,    90] loss: 0.833
[3,   100] loss: 0.816
[3,   110] loss: 0.858
[3,   120] loss: 0.808
[3,   130] loss: 0.804
[3,   140] 

In [2]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

resnet.eval()
correct = 0
total = 0
predicted_labels = []
true_labels = []

with torch.no_grad():
    for data in test_loader:
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        outputs = resnet(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        predicted_labels.extend(predicted.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

accuracy = 100 * correct / total
precision = precision_score(true_labels, predicted_labels, average='weighted')
recall = recall_score(true_labels, predicted_labels, average='weighted')
f1 = f1_score(true_labels, predicted_labels, average='weighted')

print('Accuracy on test set: %.2f %%' % accuracy)
print('Precision on test set: %.2f' % precision)
print('Recall on test set: %.2f' % recall)
print('F1-score on test set: %.2f' % f1)

Accuracy on test set: 93.43 %
Precision on test set: 0.94
Recall on test set: 0.93
F1-score on test set: 0.93


In [3]:
# Save the trained model
torch.save(resnet.state_dict(), 'resnet_model.pth')